# Sentiment Analysis - Predicting Rating Based On Review Text

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from sklearn.utils import shuffle

In [2]:
# Set up TPU instance
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.7.0
Running on TPU  ['10.17.219.50:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.17.219.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.17.219.50:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# Get dataset path on Google Colab
from google.colab import drive
drive.mount("/content/drive")
reviews_dataset_path = "drive/MyDrive/Colab Notebooks/reviews.json"

# Get RAM Info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))

if ram_gb < 20:
	print('Not using a high-RAM runtime')
else:
	print('You are using a high-RAM runtime!')

Mounted at /content/drive
Your runtime has 37.8 gigabytes of available RAM
You are using a high-RAM runtime!


In [4]:
# Read dataset into memory
review_df = pd.read_json(reviews_dataset_path, orient="records", lines=True)

In [5]:
# Shuffle Review df
review_df = shuffle(review_df, random_state=0)

# Slice into Train, Val, Test at 60:20:20
n = len(review_df)
df_train = review_df.iloc[: int(n*0.6)]
df_val = review_df.iloc[int(n*0.6) : int(n*0.8)]
df_test = review_df.iloc[int(n*0.8) :]

In [6]:
# Convert Pandas DF to TF Dataset
batch_size = 16 * tpu_strategy.num_replicas_in_sync
def convert_text_df_to_dataset(df, input_col="text", target_col="stars"):
	text_input = tf.convert_to_tensor(df[input_col], dtype=tf.string)
	target = tf.convert_to_tensor(df[target_col], dtype=tf.int8)
	dataset = tf.data.Dataset.from_tensor_slices((text_input, target))
	dataset = dataset.batch(batch_size).prefetch(AUTO)
	return dataset

train_dataset = convert_text_df_to_dataset(df_train)
val_dataset = convert_text_df_to_dataset(df_val)
test_dataset = convert_text_df_to_dataset(df_test)

In [7]:
# Create TextVectorization
max_tokens = 30000
text_vectorization = TextVectorization(max_tokens=max_tokens, output_mode="multi_hot")

# Train Vectorizer on train text
text_vectorization.adapt(df_train["text"])

In [8]:
# Vectorize Datasets
train_dataset = train_dataset.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=AUTO)
val_dataset = val_dataset.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=AUTO)
test_dataset = test_dataset.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=AUTO)

In [9]:
# Build Model
def create_model():
	inputs = keras.Input(shape=(max_tokens,))
	x = keras.layers.Dense(32, activation="relu")(inputs)
	x = keras.layers.Dropout(0.25)(x)
	x = keras.layers.Dense(16, activation="relu")(x)
	x = keras.layers.Dropout(0.25)(x)
	x = keras.layers.Dense(1)(x)
	outputs = keras.layers.ReLU(max_value=5, threshold=0)(x)

	model = keras.Model(inputs, outputs)

	model.compile(optimizer="rmsprop", loss="mean_absolute_error", metrics=["mean_squared_error"])

	return model

# Creating the model in the TPUStrategy scope means we will train the model on the TPU
with tpu_strategy.scope(): 
  model = create_model()

model_path = "models/text_vectorized.keras"
callbacks = [
	keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True),
	keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1, restore_best_weights=True)
]

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30000)]           0         
                                                                 
 dense (Dense)               (None, 32)                960032    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
 re_lu (ReLU)                (None, 1)                 0     

In [10]:
model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=callbacks)

Epoch 1/50
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


4864/4864 [==============================] - 94s 19ms/step - loss: 0.4997 - mean_squared_error: 0.8612 - val_loss: 0.4082 - val_mean_squared_error: 0.5962
Epoch 2/50
4864/4864 [==============================] - 88s 18ms/step - loss: 0.4132 - mean_squared_error: 0.6169 - val_loss: 0.3902 - val_mean_squared_error: 0.5667
Epoch 3/50
4864/4864 [==============================] - 89s 18ms/step - loss: 0.3966 - mean_squared_error: 0.5793 - val_loss: 0.3849 - val_mean_squared_error: 0.5558
Epoch 4/50
4864/4864 [==============================] - 90s 18ms/step - loss: 0.3877 - mean_squared_error: 0.5580 - val_loss: 0.3845 - val_mean_squared_error: 0.5596
Epoch 5/50
4864/4864 [==============================] - 90s 18ms/step - loss: 0.3815 - mean_squared_error: 0.5438 - val_loss: 0.3816 - val_mean_squared_error: 0.5553
Epoch 6/50
4864/4864 [==============================] - 90s 18ms/step - loss: 0.3726 - mean_squared_error: 0.5239 - val_loss: 0.3771 - val_mean_squared_error: 0.5421
Epoch 7/50
4864

In [11]:
model = keras.models.load_model(model_path)
model.evaluate(test_dataset)

1622/1622 [==============================] - 11s 7ms/step - loss: 0.3683 - mean_squared_error: 0.5285


[0.36825692653656006, 0.5284878015518188]